# **Machine Learning approach to predict real or fake tweets about disaster**
---

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

#libraries for NLP
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud,STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython.display import HTML
!pip install chart_studio
import plotly
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.figure_factory as ff
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
from collections import defaultdict
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

You should consider upgrading via the 'C:\Users\harish-4072\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


## 1.2 Reading and preparation of data

Reading [data](https://www.kaggle.com/c/nlp-getting-started/data) and choosing important columns using [pandas](https://pandas.pydata.org/)

In [2]:
data = pd.read_csv(r"D:\elggak\kaggle\Tweet Disaster Competition\nlp-getting-started\train.csv")

Displaying first 10 rows of our data using [DataFrame.head()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html)

In [3]:
data = data[['text','target']]
data.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
stemmer = SnowballStemmer("english")

def preprocess_data(data):
    
    #removal of url
    text = re.sub(r'https?://\S+|www\.\S+|http?://\S+',' ',data) 
    
    #decontraction
    text = re.sub(r"won\'t", " will not", text)
    text = re.sub(r"won\'t've", " will not have", text)
    text = re.sub(r"can\'t", " can not", text)
    text = re.sub(r"don\'t", " do not", text)    
    text = re.sub(r"can\'t've", " can not have", text)
    text = re.sub(r"ma\'am", " madam", text)
    text = re.sub(r"let\'s", " let us", text)
    text = re.sub(r"ain\'t", " am not", text)
    text = re.sub(r"shan\'t", " shall not", text)
    text = re.sub(r"sha\n't", " shall not", text)
    text = re.sub(r"o\'clock", " of the clock", text)
    text = re.sub(r"y\'all", " you all", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"n\'t've", " not have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d've", " would have", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll've", " will have", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'re", " are", text)
    
    #removal of html tags
    text = re.sub(r'<.*?>',' ',text) 
    
    # Match all digits in the string and replace them by empty string
    text = re.sub(r'[0-9]', '', text)
    text = re.sub("["
                           u"\U0001F600-\U0001F64F"  # removal of emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+",' ',text)
    
    # filtering out miscellaneous text.
    text = re.sub('[^a-zA-Z]',' ',text) 
    text = re.sub(r"\([^()]*\)", "", text)
    
    # remove mentions
    text = re.sub('@\S+', '', text)  
    
    # remove punctuations
    text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), '', text)  
    

    # Lowering all the words in text
    text = text.lower()
    text = text.split()
    
    text = [stemmer.stem(words) for words in text if words not in stopwords.words('english')]
    
    # Removal of words with length<2
    text = [i for i in text if len(i)>2] 
    text = ' '.join(text)
    return text

data["Cleaned_text"] = data["text"].apply(preprocess_data)

Displaying Cleaned Data 

In [5]:
data.head()

,text,target,Cleaned_text
0,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquak may allah forgiv
1,Forest fire near La Ronge Sask. Canada,1,forest fire near rong sask canada
2,All residents asked to 'shelter in place' are ...,1,resid ask ishelt place notifi offic evacu shel...
3,"13,000 people receive #wildfires evacuation or...",1,peopl receiv wildfir evacu order california
4,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi alaska smoke wildfir pour ...


In [6]:
common_words = ['via','like','build','get','would','one','two','feel','lol','fuck','take','way','may','first','latest'
                'want','make','back','see','know','let','look','come','got','still','say','think','great','pleas','amp']

def text_cleaning(data):
    return ' '.join(i for i in data.split() if i not in common_words)

data["Cleaned_text"] = data["Cleaned_text"].apply(text_cleaning)

# 5. Data Preprocessing 

## 5.1 Spliting original data after cleaning 

In [7]:
X_inp_clean = data['Cleaned_text']
X_inp_original = data['text']
y_inp = data['target']

Using [scikit-learn's train_test_split](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) to split the data into training and validation dataset

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_inp_clean, y_inp, test_size=0.2, random_state=42, stratify=y_inp)
y_train = np.array(y_train)
y_valid = np.array(y_valid)

checking size of data after train test split

In [9]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((6090,), (1523,), (6090,), (1523,))

In [14]:
def tfid_preprocessing(train_data, train_target):
    vectorizer = TfidfVectorizer(sublinear_tf=True)
    train_data = vectorizer.fit_transform(train_data)
    X_train, X_val, y_train, y_val = train_test_split(train_data, train_target.values, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val

def count_preprocessing(train_data, train_target):
    vectorizer = CountVectorizer()
    train_data = vectorizer.fit_transform(train_data)
    X_train, X_val, y_train, y_val = train_test_split(train_data, train_target.values, test_size=0.2, random_state=42)
    return X_train, X_val, y_train, y_val

def naive_bayes(X_train, X_val, y_train, y_val):
    nb_model = MultinomialNB()
    nb_model.fit(X_train, y_train)
    y_pred = nb_model.predict(X_val)
    print(f'Accuracy: {accuracy_score(y_val, y_pred)}')
    print(classification_report(y_val, y_pred, target_names=['ham', 'spam'], digits=6))
   

def logistic_regression(X_train, X_val, y_train, y_val):
    lr_model = LogisticRegression()
    lr_model.fit(X_train, y_train)
    y_pred = lr_model.predict(X_val)
    print(f'Accuracy: {accuracy_score(y_val, y_pred)}')
    print(classification_report(y_val, y_pred, target_names=['ham', 'spam'], digits=6))
   

def svm_classification(X_train, X_val, y_train, y_val):
    svm_model = SVC(kernel='rbf')
    svm_model.fit(X_train, y_train)
    y_pred = svm_model.predict(X_val)
    print(f'Accuracy: {accuracy_score(y_val, y_pred)}')
    print(classification_report(y_val, y_pred, target_names=['ham', 'spam'], digits=6))


def run_models(X_train, X_val, y_train, y_val, model_name):
    print(f"Running models on {model_name} dataset...")

    # Naive Bayes
    nb_model = MultinomialNB()
    nb_model.fit(X_train, y_train)
    y_pred_nb = nb_model.predict(X_val)
    print(f'Naive Bayes Accuracy: {accuracy_score(y_val, y_pred_nb)}')
    print(classification_report(y_val, y_pred_nb, target_names=['ham', 'spam'], digits=6))

    # Logistic Regression
    lr_model = LogisticRegression(C=1.0,max_iter=1000)
    lr_model.fit(X_train, y_train)
    y_pred_lr = lr_model.predict(X_val)
    print(f'Logistic Regression Accuracy: {accuracy_score(y_val, y_pred_lr)}')
    print(classification_report(y_val, y_pred_lr, target_names=['ham', 'spam'], digits=6))

    # SVM
    svm_model = SVC(kernel='rbf')
    svm_model.fit(X_train, y_train)
    y_pred_svm = svm_model.predict(X_val)
    print(f'SVM Accuracy: {accuracy_score(y_val, y_pred_svm)}')
    print(classification_report(y_val, y_pred_svm, target_names=['ham', 'spam'], digits=6))

    ensemble_model = VotingClassifier(estimators=[
        ('nb', nb_model),
        ('lr', lr_model),
        ('svm', svm_model)
    ], voting='hard')  # Hard voting for majority vote

    ensemble_model.fit(X_train, y_train)
    y_pred_ensemble = ensemble_model.predict(X_val)
    print(f'Ensemble Model Accuracy: {accuracy_score(y_val, y_pred_ensemble)}')
    print(classification_report(y_val, y_pred_ensemble, target_names=['ham', 'spam'], digits=6))


if __name__ == '__main__':
    # Run the pipeline for TF-IDF
    X_train_tfidf, X_val_tfidf, y_train_tfidf, y_val_tfidf = tfid_preprocessing(X_inp_clean,y_inp)
    run_models(X_train_tfidf, X_val_tfidf, y_train_tfidf, y_val_tfidf, "TF-IDF")

    # # Run the pipeline for Count Vectorization
    X_train_count, X_val_count, y_train_count, y_val_count = count_preprocessing(X_inp_clean,y_inp)
    run_models(X_train_count, X_val_count, y_train_count, y_val_count, "Count Vectorization")

Running models on TF-IDF dataset...
Naive Bayes Accuracy: 0.7977675640183848
              precision    recall  f1-score   support

         ham   0.792355  0.877574  0.832790       874
        spam   0.807207  0.690293  0.744186       649

    accuracy                       0.797768      1523
   macro avg   0.799781  0.783934  0.788488      1523
weighted avg   0.798684  0.797768  0.795033      1523

Logistic Regression Accuracy: 0.7892317793827971
              precision    recall  f1-score   support

         ham   0.774578  0.892449  0.829346       874
        spam   0.817829  0.650231  0.724464       649

    accuracy                       0.789232      1523
   macro avg   0.796204  0.771340  0.776905      1523
weighted avg   0.793009  0.789232  0.784652      1523

SVM Accuracy: 0.7925147734734077
              precision    recall  f1-score   support

         ham   0.770349  0.909611  0.834208       874
        spam   0.839104  0.634823  0.722807       649

    accuracy           

## 5.2 Creating function to encode data using BoW or TF-IDF

### What is BoW?   
BoW stands for "*bag of words*" which is a representation of text that describes the occurrence of words within a document.   
We just keep track of word counts and disregard the grammatical details and the word order.   
It is called a “bag” of words because any information about the order or structure of words in the document is discarded. 
The model is only concerned with whether known words occur in the document, not where in the document.
    
    
### What is TF-IDF?
TF-IDF which means Term Frequency and Inverse Document Frequency, is a scoring measure widely used in information retrieval (IR) or summarization.     
TF-IDF is intended to reflect how relevant a term is in a given document. It is a technique in Natural Language Processing for converting words in Vectors and with some semantic information and it gives weighted to uncommon words , used in various NLP applications.    

For [BoW](https://www.mygreatlearning.com/blog/bag-of-words/) approach we use scikit-learn's [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) and for [TF-IDF](https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76) we use [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [10]:
def encoding(train_data,valid_data,bow=False,n=1,tf_idf=False):
    if bow==True:
        cv = CountVectorizer(ngram_range=(n,n))
        cv_df_train = cv.fit_transform(train_data).toarray()
        train_df = pd.DataFrame(cv_df_train,columns=cv.get_feature_names())
        cv_df_valid = cv.transform(valid_data).toarray()
        valid_df = pd.DataFrame(cv_df_valid,columns=cv.get_feature_names())
        
    elif tf_idf==True:
        
        tfidf = TfidfVectorizer(ngram_range=(n, n), use_idf=1,smooth_idf=1,sublinear_tf=1)    
        tf_df_train = tfidf.fit_transform(train_data).toarray()
        train_df = pd.DataFrame(tf_df_train,columns=tfidf.get_feature_names())
        tf_df_valid = tfidf.transform(valid_data).toarray()
        valid_df = pd.DataFrame(tf_df_valid,columns=tfidf.get_feature_names())
        
    return train_df,valid_df 

## 5.3 Encoding training and validation data

We encode our data in all possible combinations provided by our function

In [11]:
X_train_bow1 , X_valid_bow1 = encoding(X_train,X_valid,bow=True) 
X_train_bow2 , X_valid_bow2 = encoding(X_train,X_valid,bow=True,n=2) 
X_train_bow3 , X_valid_bow3 = encoding(X_train,X_valid,bow=True,n=3) 
X_train_tfidf1 , X_valid_tfidf1 = encoding(X_train,X_valid,tf_idf=True) 
X_train_tfidf2 , X_valid_tfidf2 = encoding(X_train,X_valid,tf_idf=True,n=2) 
X_train_tfidf3 , X_valid_tfidf3 = encoding(X_train,X_valid,tf_idf=True,n=3)

AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'

# 6. Training and tuning Machine Learining Models

### What is a classification report?
A Classification report is used to measure the quality of predictions from a classification algorithm.   
The report shows the main classification metrics precision, recall and f1-score on a per-class basis. The metrics are calculated by using true and false positives, true and false negatives. Positive and negative in this case are generic names for the predicted classes. 
### What is a confusion matrix?
A confusion matrix is a table that is often used to describe the performance of a classification model (or "classifier") on a set of test data for which the true values are known. The confusion matrix itself is relatively simple to understand, but the related terminology can be confusing.  

#### In a confusion matrix there are 4 basic terminologies :
* **true positives (TP)** : We predicted yes (they are real tweets), and they are actually real.
* **true negatives (TN)** : We predicted no, and they are fake.
* **false positives (FP)**: We predicted yes, but they are't actually real. (Also known as a "Type I error.")
* **false negatives (FN)**: We predicted no, but they are real. (Also known as a "Type II error.")

Now let's create functions to display model's [classification report](https://datascience.stackexchange.com/questions/64441/how-to-interpret-classification-report-of-scikit-learn) and [confusion matrix](https://machinelearningmastery.com/confusion-matrix-machine-learning/)

In [ ]:
def c_report(y_true,y_pred):
    print("Classifictaion Report")
    print(classification_report(y_true, y_pred))
    acc_scr = accuracy_score(y_true, y_pred)
    print("Accuracy : "+ str(acc_scr))
    return acc_scr

def plot_cm(y_true,y_pred,cmap = "Blues"):
    mtx = confusion_matrix(y_true, y_pred)
    sns.heatmap(mtx, annot = True, fmt='d', linewidth=0.5,
               cmap=cmap, cbar = False)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')

## 6.1 Logistic Regression

### About Logistic Regression   

Logistic regression is a statistical model that in its basic form uses a logistic function to model a binary dependent variable, although many more complex extensions exist. In regression analysis, logistic regression (or logit regression) is estimating the parameters of a logistic model (a form of binary regression).

Logistic regression is the appropriate regression analysis to conduct when the dependent variable is dichotomous (binary).  Like all regression analyses, the logistic regression is a predictive analysis.  Logistic regression is used to describe data and to explain the relationship between one dependent binary variable and one or more nominal, ordinal, interval or ratio-level independent variables.

Now let's create a [Logistic Regression](https://medium.com/analytics-vidhya/applying-text-classification-using-logistic-regression-a-comparison-between-bow-and-tf-idf-1f1ed1b83640) model and train it.

In [ ]:
model_bow1_logreg = LogisticRegression()
model_bow1_logreg.fit(X_train_bow1,y_train)
pred_bow1_logreg = model_bow1_logreg.predict(X_valid_bow1)

Printing classification report and ploting confusion matrix for the predictions made by LogisticRegression(BoW,n-grams=1) model

In [ ]:
acc_bow1_logreg = c_report(y_valid,pred_bow1_logreg)
plot_cm(y_valid,pred_bow1_logreg)

Now training another Logistic Regression model with n-grams=2 and BoW 

In [ ]:
model_bow2_logreg = LogisticRegression()
model_bow2_logreg.fit(X_train_bow2,y_train)
pred_bow2_logreg = model_bow2_logreg.predict(X_valid_bow2)

Printing classification report and ploting confusion matrix for the predictions made by LogisticRegression(BoW,n-grams=2) model

In [ ]:
acc_bow2_logreg = c_report(y_valid,pred_bow2_logreg)
plot_cm(y_valid,pred_bow2_logreg)

We can observe that as n is increasing model accuracy is decreasing   
let's try to increase n one last time just to be sure

In [ ]:
model_bow3_logreg = LogisticRegression()
model_bow3_logreg.fit(X_train_bow3,y_train)
pred_bow3_logreg = model_bow3_logreg.predict(X_valid_bow3)

Printing classification report and ploting confusion matrix for the predictions made by LogisticRegression(BoW,n-grams=3) model

In [ ]:
acc_bow3_logreg = c_report(y_valid,pred_bow3_logreg)
plot_cm(y_valid,pred_bow3_logreg)

From the above results it's clear that using n = 1 will always give us more accuray,  
now let's use tfidf approach with n = 1 to train our Logistic Regression model

In [ ]:
model_tfidf1_logreg = LogisticRegression(C=1.0)
model_tfidf1_logreg.fit(X_train_tfidf1,y_train)
pred_tfidf1_logreg = model_tfidf1_logreg.predict(X_valid_tfidf1)

Printing classification report and ploting confusion matrix for the predictions made by LogisticRegression(TF-IDF,n-grams=1) model

In [ ]:
acc_tfidf1_logreg = c_report(y_valid,pred_tfidf1_logreg)
plot_cm(y_valid,pred_tfidf1_logreg)

From Logistic Regression we saw n-grams = 1 gives the best results 

## 6.2 Multinomial Naive Bayes

### About Multinomial Naive Bayes  

Multinomial Naive Bayes algorithm is a probabilistic learning method that is mostly used in Natural Language Processing (NLP). The algorithm is based on the Bayes theorem and predicts the tag of a text such as a piece of email or newspaper article. It calculates the probability of each tag for a given sample and then gives the tag with the highest probability as output.

Naive Bayes classifier is a collection of many algorithms where all the algorithms share one common principle, and that is each feature being classified is not related to any other feature. The presence or absence of a feature does not affect the presence or absence of the other feature.
    
Now let's create [MultinomialNB](https://www.upgrad.com/blog/multinomial-naive-bayes-explained/) model and train it.

In [ ]:
model_bow1_NB = MultinomialNB(alpha=0.7)
model_bow1_NB.fit(X_train_bow1,y_train)
pred_bow1_NB = model_bow1_NB.predict(X_valid_bow1)

Printing classification report and ploting confusion matrix  for the predictions of MultinomialNB(BoW,n-grams=1) model

In [ ]:
acc_bow1_NB = c_report(y_valid,pred_bow1_NB)
plot_cm(y_valid,pred_bow1_NB)

Creating a MultinomialNB model and training it with TF-IDF approach

In [ ]:
model_tfidf1_NB = MultinomialNB(alpha=0.7)
model_tfidf1_NB.fit(X_train_tfidf1,y_train)
pred_tfidf1_NB = model_tfidf1_NB.predict(X_valid_tfidf1)

Printing classification report and ploting confusion matrix for the predictions of MultinomialNB(TF-IDF,n-grams=1) model

In [ ]:
acc_tfidf1_NB = c_report(y_valid,pred_tfidf1_NB)
plot_cm(y_valid,pred_tfidf1_NB)

## 6.3 Random Forest Classifier

### About Random Forest Classifier   

Random forests is a supervised learning algorithm. It can be used both for classification and regression. It is also the most flexible and easy to use algorithm. A forest is comprised of trees. It is said that the more trees it has, the more robust a forest is. Random forests creates decision trees on randomly selected data samples, gets prediction from each tree and selects the best solution by means of voting. It also provides a pretty good indicator of the feature importance.

Random forests has a variety of applications, such as recommendation engines, image classification and feature selection. It can be used to classify loyal loan applicants, identify fraudulent activity and predict diseases. It lies at the base of the Boruta algorithm, which selects important features in a dataset.   

Now let's create a [RandomForestClassifier](https://www.geeksforgeeks.org/random-forest-classifier-using-scikit-learn/) model and train it.

In [ ]:
model_tfidf1_RFC = RandomForestClassifier()
model_tfidf1_RFC.fit(X_train_tfidf1,y_train)
pred_tfidf1_RFC = model_tfidf1_RFC.predict(X_valid_tfidf1)

Printing classification report and ploting confusion matrix for predictions of RandomForestClassifier model

In [ ]:
acc_tfidf1_RFC = c_report(y_valid,pred_tfidf1_RFC)
plot_cm(y_valid,pred_tfidf1_RFC)

## 6.4 eXtreme Gradient Boosting Classifier

### About XGBClassifier
The XGBoost stands for eXtreme Gradient Boosting, which is a boosting algorithm based on gradient boosted decision trees algorithm.     
XGBoost applies a better regularization technique to reduce overfitting, and it is one of the differences from the gradient boosting.    

Now let's create a [XGBClassifier](https://machinelearningmastery.com/develop-first-xgboost-model-python-scikit-learn/) model and train it.

In [ ]:
model_tfidf1_XGB = XGBClassifier(eval_metric='mlogloss')
model_tfidf1_XGB.fit(X_train_tfidf1,y_train)
pred_tfidf1_XGB = model_tfidf1_XGB.predict(X_valid_tfidf1)

Printing classification report and ploting confusion matrix for the predictions made by the XGBClassifier model

In [ ]:
acc_tfidf1_XGB = c_report(y_valid,pred_tfidf1_XGB)
plot_cm(y_valid,pred_tfidf1_XGB)

## 6.5 CatBoostClassifier

### About CatBoostClassifier

CatBoost is a recently open-sourced machine learning algorithm from Yandex. It can easily integrate with deep learning frameworks like Google’s TensorFlow and Apple’s Core ML. It can work with diverse data types to help solve a wide range of problems that businesses face today. To top it up, it provides best-in-class accuracy.    

It yields state-of-the-art results without extensive data training typically required by other machine learning methods. 
   
“Boost” comes from gradient boosting machine learning algorithm as this library is based on gradient boosting library.      
Gradient boosting is a powerful machine learning algorithm that is widely applied to multiple types of business challenges like fraud detection, recommendation items, forecasting and it performs well also. It can also return very good result with relatively less data, unlike DL models that need to learn from a massive amount of data.


Now let's create a [CatBoostClassifier](https://catboost.ai/docs/concepts/python-reference_catboostclassifier.html) model with 100 iterations and training it

In [ ]:
model_tfidf1_CBC = CatBoostClassifier(iterations=100)
model_tfidf1_CBC.fit(X_train_tfidf1,y_train)
pred_tfidf1_CBC = model_tfidf1_CBC.predict(X_valid_tfidf1)


Printing classification report and ploting confusion matrix for the predictions made by the above model

In [ ]:
acc_tfidf1_CBC = c_report(y_valid,pred_tfidf1_CBC)
plot_cm(y_valid,pred_tfidf1_CBC)

## 6.6 Support Vector CLassifier 

### About SVC
SVM offers very high accuracy compared to other classifiers such as logistic regression, and decision trees. It is known for its kernel trick to handle nonlinear input spaces. It is used in a variety of applications such as face detection, intrusion detection, classification of emails, news articles and web pages, classification of genes, and handwriting recognition.

SVM is an exciting algorithm and the concepts are relatively simple. The classifier separates data points using a hyperplane with the largest amount of margin. That's why an SVM classifier is also known as a discriminative classifier. SVM finds an optimal hyperplane which helps in classifying new data points.   

Now let's create a [SVC](https://pythonprogramming.net/linear-svc-example-scikit-learn-svm-python/) model and training it

In [ ]:
model_tfidf1_SVC = SVC(kernel='linear', degree=3, gamma='auto')
model_tfidf1_SVC.fit(X_train_tfidf1,y_train)
pred_tfidf1_SVC = model_tfidf1_SVC.predict(X_valid_tfidf1)

Printing classification report and ploting confusion matrix for the SVC model

In [ ]:
acc_tfidf1_SVC = c_report(y_valid,pred_tfidf1_SVC)
plot_cm(y_valid,pred_tfidf1_SVC)

## 6.7 Voting Classifier

### About Voting Classifier

A Voting Classifier is a machine learning model that trains on an ensemble of numerous models and predicts an output (class) based on    
their highest probability of chosen class as the output. It simply aggregates the findings of each classifier passed into Voting Classifier    
and predicts the output class based on the highest majority of voting. The idea is instead of creating separate dedicated models and finding the accuracy for each them, we create a single model which trains by these models and predicts output based on their combined majority of voting for each output class.

#### Voting Classifier supports two types of votings :  


* **Hard Voting** : In hard voting, the predicted output class is a class with the highest majority of votes i.e the class    which had the highest probability of being predicted by each of the classifiers. Suppose three classifiers predicted the output class(A, A, B), so here the majority predicted A as output. Hence A will be the final prediction.


* **Soft Voting** : In soft voting, the output class is the prediction based on the average of probability given to that class. Suppose given some input to three models, the prediction probability for class A = (0.30, 0.47, 0.53) and B = (0.20, 0.32, 0.40). So the average for class A is 0.4333 and B is 0.3067, the winner is clearly class A because it had the highest probability averaged by each classifier
 
 
Now let's create a [VotingClassifier](https://www.geeksforgeeks.org/ml-voting-classifier-using-sklearn/) with soft voting and train it 

In [ ]:
estimators = []
estimators.append(('LR', 
                  LogisticRegression()))
estimators.append(('NB', MultinomialNB(alpha=0.7)))
estimators.append(('XBG', XGBClassifier(eval_metric='mlogloss')))

model_tfidf1_VC = VotingClassifier(estimators=estimators,voting='soft')
model_tfidf1_VC.fit(X_train_tfidf1,y_train)
pred_tfidf1_VC = model_tfidf1_VC.predict(X_valid_tfidf1)

Printing classification report and ploting confusion matrix for VotingClasssifier

In [ ]:
acc_tfidf1_VC = c_report(y_valid,pred_tfidf1_VC)
plot_cm(y_valid,pred_tfidf1_VC,cmap = "Greens")

# 7. Comparing the Accuracy of all models

In [ ]:
results = pd.DataFrame([["Logistic Regression BoW1",acc_bow1_logreg],["Logistic Regression BoW2",acc_bow2_logreg],
                       ["Logistic Regression BoW3",acc_bow3_logreg],["Logistic Regression Tf-Idf1",acc_tfidf1_logreg],
                       ["Naive Bayes Tf-Idf1",acc_tfidf1_NB],["Random Forest Tf-Idf1",acc_tfidf1_RFC],
                       ["XGBClassifier Tf-Idf1",acc_tfidf1_XGB],["CatBoost Tf-Idf1",acc_tfidf1_CBC],
                        ["SVC Tf-Idf1",acc_tfidf1_SVC],["Voting Tf-Idf1",acc_tfidf1_VC]],
                       columns = ["Models","Accuracy Score"]).sort_values(by='Accuracy Score',ascending=False)

results.style.background_gradient(cmap='Blues')

# 8. Conclusion

Among all Simple classification models used above Voting Classifier performed best with tf-idf and ngrams = 1